# Import necessary libraries

In [4]:
import sys
import os
import glob
import PIL
from PIL import Image
import numpy as np

# Set input and output image folders

In [ ]:
archetypes_folder = 'insert/folder/with/input/heightmaps/here'
daylightMaps_folder = 'insert/folder/with/input/daylight/maps/here'

# Load images

In [13]:
archetypes = glob.glob(archetypes_folder + '/*.png', recursive=False)
archetypes.sort(key=solar_chars) #lambda x: os.path.getmtime(x))

In [14]:
daylightMaps = glob.glob(daylightMaps_folder + '/*.png', recursive=False)
daylightMaps.sort(key=solar_chars) #lambda x: os.path.getmtime(x))

In [16]:
len(archetypes), len(daylightMaps)

(1374, 1401, 1401)

# Make sure only images with valid pairs are included

In [19]:
arch_ids = []
for image in archetypes:
    arch_ids.append(image.split('\\')[-1].split('.png')[0])

In [20]:
day_ids = []
for image in daylightMaps:
    day_ids.append(image.split('\\')[-1].split('.png')[0])

In [22]:
both = set(arch_ids).intersection(day_ids)

indices_A = [arch_ids.index(x) for x in both]
indices_B = [day_ids.index(x) for x in both]

In [23]:
len(indices_A), len(indices_B)

(1374, 1374, 1374)

# Create folders to store your paired dataset

In [24]:
save_folder = 'your/location/here'

paths = [folder + '/train', folder + '/val', folder + '/test']

for path in paths:
    try:
        os.makedirs(path)
    except OSError:
        print ("Creation of the directory %s failed" % path)
    else:
        print ("Successfully created the directory %s" % path)

# Get train, test, and validation split

In [25]:
# Split in training, test, and validation sets

total_imgs = len(indices_A)
test_pct = 0.1 # 10% for test set
val_pct = 0.01 # 1% for valudation

indices = np.arange(0, total_imgs, 1)
rand_idx = np.random.permutation(indices)

test_cut = int(test_pct*total_imgs)
val_cut = int(total_imgs*val_pct)
cut = test_cut + val_cut

train_idx = np.setdiff1d(indices, rand_idx[:cut])
test_idx = rand_idx[0:test_cut]
val_idx = rand_idx[test_cut:cut]

In [27]:
for i in range(len(indices_A)):
    im_list = [archetypes[indices_A[i]], daylightMaps[indices_B[i]]]
    imgs = [ PIL.Image.open(i) for i in im_list ]
    min_shape = sorted( [(np.sum(i.size), i.size ) for i in imgs])[0][1]
    #horizontal stack
    imgs_comb = np.hstack( (np.asarray( i.resize(min_shape) ) for i in imgs ) )
    imgs_comb = PIL.Image.fromarray( imgs_comb)
    if i in train_idx:
        imgs_comb.save( '{}.png'.format(save_folder + 'train\\' + archetypes[i].split('\\')[-1][:-4]) )
    elif i in test_idx:
        imgs_comb.save( '{}.png'.format(save_folder + 'test\\' + archetypes[i].split('\\')[-1][:-4]) )
    else:
        imgs_comb.save( '{}.png'.format(save_folder + 'val\\' + archetypes[i].split('\\')[-1][:-4]) )